<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [11]:
# Загрузим все необходимые библиотеки

import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import spacy

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

import re
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import xgboost as xgb
from nltk.corpus import stopwords 
from sklearn.metrics import f1_score

from time import time
from tqdm.notebook import tqdm

pd.options.display.max_columns = None # полностью отображаем столбцы

In [12]:


pth1 = '/datasets/toxic_comments.csv'
pth2 = 'toxic_comments.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [13]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [15]:
data = df.drop(columns='Unnamed: 0')

In [16]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanzl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stop_words = set(stopwords.words('english'))

In [18]:
lemmatizer = WordNetLemmatizer()

In [19]:
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

sentence = "The striped bats are hanging on their feet for best"

# Parse the sentence using the loaded 'en' model object `nlp`
doc = nlp(sentence)

# Extract the lemma for each token and join
" ".join([token.lemma_ for token in doc])
#> 'the strip bat be hang on -PRON- foot for good'

'the stripe bat be hang on their foot for good'

In [20]:
def predproc(text):
    text = re.sub(r'[^a-zA-z ]', ' ', text)
    text = text.lower()
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    text = ' '.join(text.split())
    return text

In [21]:
tqdm.pandas()

data['text_ready'] = data['text'].progress_apply(predproc)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [22]:
data.head()

,text,toxic,text_ready
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Вывод: был загружен датасет и обработан текст

## Обучение

In [23]:
train_features, test_features, train_target, test_target = train_test_split(
    data.drop(['toxic','text'], axis=1),
    data['toxic'],
    test_size=0.1,
    random_state=12345,
)

In [24]:
print('Размер тренировочного датафрейма',train_features.shape)
print('Размер тестового датафрейма',test_features.shape)

Размер тренировочного датафрейма (143362, 1)
Размер тестового датафрейма (15930, 1)


In [25]:
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(train_features['text_ready']) 
tf_idf_test = count_tf_idf.transform(test_features['text_ready'])

print("Размер тренировочной матрицы:", tf_idf_train.shape)
print("Размер тестовой матрицы:", tf_idf_test.shape)

Размер тренировочной матрицы: (143362, 145126)
Размер тестовой матрицы: (15930, 145126)


In [26]:
results = pd.DataFrame({
    'model':[],
    'f1_on RS_CV':[],
    'best_param':[]
})

def train_model(model, parameters):
    
    model_random = GridSearchCV(
        model,
        parameters,
        scoring='f1'
    )
    
    model_random.fit(tf_idf_train,train_target)
    # высчитаем метрики
    f1 = model_random.best_score_
    
    print('Best parameters:', model_random.best_params_)
    print('F1:', f1)
    
    results.loc[ len(results.index )] = [model_random.best_estimator_,f1,model_random.best_params_]

In [27]:
lr_param = {
    "C" : [10,15,20]
}

lr = LogisticRegression(max_iter=1000)

lr_random = train_model(lr, lr_param)

Best parameters: {'C': 15}
F1: 0.7825615764370648


In [28]:
dec_tree_param = {
    "max_depth": [5,20,40,60]
}

dec_tree = DecisionTreeClassifier()

train_model(dec_tree, dec_tree_param)

Best parameters: {'max_depth': 60}
F1: 0.7100266100909136


In [29]:
rand_for_param = {
    "n_estimators" : [5,10,15],
     "max_depth" : [50,150,300]
}

rand_for = RandomForestClassifier()

train_model(rand_for,rand_for_param)

Best parameters: {'max_depth': 300, 'n_estimators': 10}
F1: 0.5386245214993396


In [30]:
lgbm_param = {
    'max_depth': [15, 25, 50],
    'learning_rate': [0.1, 0.3]
}

lgbm = lgb.LGBMClassifier(
    boosting_type='gbdt',
    n_jobs=-1
)

train_model(lgbm, lgbm_param)

Best parameters: {'learning_rate': 0.3, 'max_depth': 50}
F1: 0.7785038455854544


In [31]:
results

,model,f1_on RS_CV,best_param
0,"LogisticRegression(C=15, max_iter=1000)",0.782562,{'C': 15}
1,DecisionTreeClassifier(max_depth=60),0.710027,{'max_depth': 60}
2,"(DecisionTreeClassifier(max_depth=300, max_fea...",0.538625,"{'max_depth': 300, 'n_estimators': 10}"
3,"LGBMClassifier(learning_rate=0.3, max_depth=50)",0.778504,"{'learning_rate': 0.3, 'max_depth': 50}"


Вывод: Лучший результат f1_score на обучении показала модель LogisticRegression(C=15, max_iter=1000), поэтому выберем её как лучшую.

In [33]:
lr = LogisticRegression(C=15, max_iter=1000)
lr.fit(tf_idf_train,train_target)
print('F1 on test:',f1_score(lgbm.predict(tf_idf_test),test_target))

F1 on test: 0.7914364640883977


## Выводы

Общий вывод: 
1. Был загружен и обработан датасет
2. Было обучено 4 модели (LR,DT,RF,LGBM), 3 из 4-х моделей показали удовлетворительный результат
3. Лучшей моделью была выбаран модель LogisticRegression(C=15, max_iter=1000)
4. На тестовой выборке LogisticRegression(C=15, max_iter=1000) показала f1_score = 0.79, что можно считать хорошим результатом


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны